# Brazilian State Scrapper

In [1]:
from plugins.selenium import Selenium
from selenium.webdriver.common.by import By
import pandas as pd
import string

## Populating queue

In [ ]:
webdriver = Selenium()
webdriver.start()
webdriver.go_to_url("https://inanyplace.blogspot.com/2017/01/lista-de-estados-brasileiros-sigla-estado-capital-e-regiao.html")

driver = webdriver.get_driver()

### Extract table values

In [6]:
table   = driver.find_element(By.XPATH, '//table[@bgcolor="#ffffff"]')
columns = [header.text for header in table.find_elements(By.XPATH, '//th/span')]
rows    = [row.find_elements(By.XPATH, './td/span') for row in table.find_elements(By.XPATH, '//tbody/tr')]
values  = [[cell.text for cell in row] for row in rows if row]

df = pd.DataFrame(values, columns=columns)[['Estado', 'Capital', 'Região']]
df.columns = ['estado', 'capital', 'regiao']
df = df.map(lambda x: x.title() if isinstance(x, str) else x)
df.head(3)

,estado,capital,regiao
0,Acre,Rio Branco,Norte
1,Pará,Belém,Norte
2,Rondônia,Porto Velho,Norte


### Extract file values

In [9]:
df_capital_populacao = pd.read_excel("../input/PopulaçãoxCapital.xlsx")
df_capital_populacao.drop_duplicates(inplace=True)
df_capital_populacao[['capital', 'populacao']] = df_capital_populacao['Capital/populacao'].str.split(':', expand=True)
df_capital_populacao.drop(columns=['Capital/populacao'], inplace=True)
df_capital_populacao['populacao'] = df_capital_populacao['populacao'].str.replace(f"[{string.punctuation}]", "", regex=True).astype(int)
df_capital_populacao.head(3)

,capital,populacao
0,Aracajú,628849
1,Belém,1398531
2,Belo Horizonte,2416339


### Join informations

In [10]:
df_inner = pd.merge(df, df_capital_populacao, on='capital', how='inner')
df_inner.head(3)

,estado,capital,regiao,populacao
0,Acre,Rio Branco,Norte,387852
1,Pará,Belém,Norte,1398531
2,Rondônia,Porto Velho,Norte,514873
